<a href="https://colab.research.google.com/github/jonatansg/CEIABD/blob/main/Actividad_2_1_An%C3%A1lisis_de_observaciones_influyentes.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **Actividad 2.1 – Análisis de observaciones influyentes**






## **1. Observaciones influyentes**


In [133]:
import numpy as np
import pandas as pd
import scipy.stats as st
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import MinMaxScaler    # Escalamiento por rangos
from sklearn.preprocessing import StandardScaler  # Escalamiento Z

In [134]:
# Generamos (100 elementos) ingresos uniformemente distribuidos en [15000,25000]
datos = 15000+np.random.rand(100)*10000
datos_ord = datos
datos_ord.sort()
datos

array([15109.95828658, 15183.90676547, 15224.6930832 , 15265.32203874,
       15311.83135061, 15469.65966775, 15514.25813484, 15751.37781736,
       15827.98667925, 15915.82073327, 16065.98253134, 16177.5108289 ,
       16368.18630919, 16368.85630069, 16393.56074073, 16401.13175766,
       16662.83368756, 16716.79285848, 16753.71278623, 16814.3543509 ,
       17064.21271761, 17078.86255146, 17149.40378591, 17453.49109681,
       17464.02033239, 17578.87685433, 17594.23433431, 17622.640443  ,
       17680.63640823, 17686.77359385, 17721.4513723 , 17723.07326519,
       17740.55220687, 17886.93736771, 17944.65759542, 17971.21715623,
       18342.43890817, 18418.17351698, 18556.72716465, 18609.7389694 ,
       18796.92408167, 18831.38528249, 18892.95614042, 19094.11652191,
       19187.70524892, 19214.24550592, 19252.13504469, 19348.27338604,
       19361.26529135, 19460.05772958, 19760.50402199, 19874.3119825 ,
       19954.91316206, 19963.66247201, 20189.17705283, 20256.90182303,
      

### **a) Calcular la media y la mediana antes de realizar la modificación de incluir unos ingresos de 500.000€**



In [135]:
media=  np.mean(datos)
mediana=  np.median (datos) # Es el valor central cuando los datos se ordenan
print(f" Media= {media} y Mediana={mediana}")

 Media= 19711.90388386042 y Mediana=19610.280875785276


In [136]:
# Simulamos (porque se lo asignamos en la siguiente línea) que el vecino 50 tiene unos ingresos significativamente mayores que el resto
datos[50]=500000
datos

array([ 15109.95828658,  15183.90676547,  15224.6930832 ,  15265.32203874,
        15311.83135061,  15469.65966775,  15514.25813484,  15751.37781736,
        15827.98667925,  15915.82073327,  16065.98253134,  16177.5108289 ,
        16368.18630919,  16368.85630069,  16393.56074073,  16401.13175766,
        16662.83368756,  16716.79285848,  16753.71278623,  16814.3543509 ,
        17064.21271761,  17078.86255146,  17149.40378591,  17453.49109681,
        17464.02033239,  17578.87685433,  17594.23433431,  17622.640443  ,
        17680.63640823,  17686.77359385,  17721.4513723 ,  17723.07326519,
        17740.55220687,  17886.93736771,  17944.65759542,  17971.21715623,
        18342.43890817,  18418.17351698,  18556.72716465,  18609.7389694 ,
        18796.92408167,  18831.38528249,  18892.95614042,  19094.11652191,
        19187.70524892,  19214.24550592,  19252.13504469,  19348.27338604,
        19361.26529135,  19460.05772958, 500000.        ,  19874.3119825 ,
        19954.91316206,  

### **b) Aplicar el método de Probabilidad global, para detectar los outliers**

In [137]:
# CRITERIO 1: PROBABILIDAD GLOBAL
# Asumiendo que las variables tiene una distribución normal.
# Probabilidad de la muestra de estar dentro de las bandas
p_g=0.95
# probabilidad global
alfa_g=(1-p_g)/2
# probabilidad  para un solo dato
alfa= 1-(1-alfa_g)**(1/len(datos)) # Se realiza este ajuste para ser más precisos.
#alfa = alfa_g # Si no realiza la corrección de la línea anterior entonces hubiese detectado más valores que son outliers cuando realmente no lo son. Descomentar esta línea para ver el resultado, que es el mismo que el obtenidoi con el método 2 de este código.
                # con esta corrección lo que se pretende es obtener bandas lo sificientemente anchas.

In [138]:
## CRITERIO 2:  Criterio Chauvenet
# El uso del Criterio de Chauvenet es controvertido en algunas áreas de la estadística, ya que puede llevar a descartar datos válidos,
# especialmente en conjuntos de datos pequeños o en situaciones donde los datos no siguen una distribución normal.
# Por lo tanto, debe usarse con precaución y teniendo en cuenta el contexto específico de los datos analizados.

#alfa=1/(2*len(datos))
Z_alfa=st.norm.ppf(1-alfa/2)


In [139]:
# Impresión de resultados
alfa=round(alfa,5)
Z_alfa=round(Z_alfa,5)
print(f" Alfa ={alfa}")
print(f" Z_alfa ={Z_alfa}")

 Alfa =0.00025
 Z_alfa =3.65906


In [140]:
# Utilizaremos el Criterio 1 ó el Criterio 2 para calcular alfa y Z-alfa en función de la naturaleza de los datos que estamos procesando.
xL=  round(np.mean(datos) - Z_alfa * np.std(datos),4)
xU=  round(np.mean(datos) + Z_alfa * np.std(datos),4)
print(f" Banda= [ {xL},{xU}]")

 Banda= [ -150638.9646,199667.5623]


In [141]:
for i in  range(len(datos)):
        if datos[i] < xL  or datos[i]>xU:
          print(f" El dato[{i}]={datos[i]} es un outlier")

 El dato[50]=500000.0 es un outlier


### **c) Repetir el mismo procedimiento de detectar los outliers para la mediana: ¿Qué ocurre?**

In [142]:
xL_median= round(np.median(datos) - Z_alfa * np.std(datos),4)
xU_median= round(np.median(datos) + Z_alfa * np.std(datos),4)

# Detectar outliers basados en la mediana
for i in range(len(datos)):
    if datos[i] < xL_median or datos[i] > xU_median:
        print(f" El dato[{i}]={datos[i]} es un outlier (según mediana)")


 El dato[50]=500000.0 es un outlier (según mediana)


La mediana es menos sensible a los valores extremos. Este cambio debería hacer que el número de outliers detectados sea diferente al usar la media, y podría reducir el número de outliers detectados si la distribución tiene valores extremos que afectan la media. Aunque en este ejercicio el outlier es el mismo.

## **2. Escalamiento de datos**

Considerar que la variable X toma los valores 1,2,3,4,5,6,7,8,9,10.

### **a) ¿Cuánto vale la media, mediana, la desviación estándar muestral, la varianza muestral y el rango de la variable X?**

In [143]:
# Variable X (1 a 10)
X = np.array([1, 2, 3, 4, 5, 6, 7, 8, 9, 10])

media_X = np.mean(X)
mediana_X = np.median(X)
desviacion_estandar_X = np.std(X, ddof=1)
varianza_X = np.var(X, ddof=1)
rango_X = np.ptp(X)

# Mostrar los resultados
print("Media:", media_X)
print("Mediana:", mediana_X)
print("Desviación estándar muestral:", desviacion_estandar_X)
print("Varianza muestral:", varianza_X)
print("Rango:", rango_X)

Media: 5.5
Mediana: 5.5
Desviación estándar muestral: 3.0276503540974917
Varianza muestral: 9.166666666666666
Rango: 9


### **b) Utilizar la función describe() de Panda, para obtener la media, desviación estándar, etc…**

In [144]:
print(pd.DataFrame(X).describe())

              0
count  10.00000
mean    5.50000
std     3.02765
min     1.00000
25%     3.25000
50%     5.50000
75%     7.75000
max    10.00000


### **c) ¿Por qué el resultado de calcular la desviación estándar con Numpy es diferente a la calculada por describe de Panda? ¿Qué ajuste sería necesario realizar para que los resultados fuesen similares/iguales?**

La diferencia entre el cálculo de la desviación estándar utilizando NumPy y Pandas se debe al ajuste de grados de libertad que se utiliza en cada caso.

En NumPy, cuando calculas la desviación estándar con np.std(), por defecto se utiliza el denominador 𝑁, donde 𝑁 es el número total de elementos. Esto se refiere a la desviación estándar de la población.

La función describe() de Pandas usa por defecto el denominador 𝑁 − 1 (con el parámetro ddof=1), que es el ajuste muestral. Esto se hace para corregir el sesgo que puede ocurrir al estimar la desviación estándar de una población completa a partir de una muestra. Esta corrección se llama Bessel's correction.

Para que los resultados de la desviación estándar calculados con NumPy y Pandas sean iguales, se debe ajustar el parámetro ddof en NumPy para que use
𝑁 − 1 como denominador, al igual que Pandas.

In [145]:
# Cálculo con numpy (muestral)
std_numpy_muestral = np.std(X, ddof=1)

# Cálculo con pandas
std_pandas = pd.Series(X).std()

print(f"Desviación estándar con Numpy (muestral): {std_numpy_muestral}")
print(f"Desviación estándar con Pandas: {std_pandas}")

Desviación estándar con Numpy (muestral): 3.0276503540974917
Desviación estándar con Pandas: 3.0276503540974917


### **d) Estandarizar la variable (escalamiento) mediante rangos y a continuación calcular la media y la mediana de la variable escalada**

In [146]:
Data = pd.DataFrame(X, columns=['X'])

# Escalamiento
# ==============================================================================
# Selección de columnas numéricas
numeric_cols = Data.select_dtypes(include=['float64', 'int']).columns.to_list()

# MinMaxScaler - Estandarización por rangos
preprocessor_RR = ColumnTransformer(
                   [('scale', MinMaxScaler(), numeric_cols)],remainder='passthrough')

Data_prep_RR = preprocessor_RR.fit_transform(Data)

# Creación de DataFrame
Data_scaled_Range = pd.DataFrame(Data_prep_RR, columns=numeric_cols)

# Cálculo de media y mediana de las variables escaladas
media_Range = Data_scaled_Range.mean()
mediana_Range = Data_scaled_Range.median()

# Resultados
print(f"Media con escalamiento por rangos: {media_Range}")
print(f"Mediana con escalamiento por rangos: {mediana_Range}")

Media con escalamiento por rangos: X    0.5
dtype: float64
Mediana con escalamiento por rangos: X    0.5
dtype: float64


### **e) Repetir el apartado anterior con el escalamiento Z - score**

In [147]:
# Standard scaler (Z-score)
preprocessor_SS = ColumnTransformer(
    [('scale', StandardScaler(), numeric_cols)], remainder='passthrough'
)
Data_prep_SS = preprocessor_SS.fit_transform(Data)

# Creación de DataFrame
Data_scaled_Z = pd.DataFrame(Data_prep_SS, columns=numeric_cols)

# Cálculo de media y mediana de las variables escaladas
media_Z = Data_scaled_Z.mean()
mediana_Z = Data_scaled_Z.median()

# Resultados
print(f"Media con escalamiento Z-score: {media_Z}")
print(f"Mediana con escalamiento Z-score: {mediana_Z}")

Media con escalamiento Z-score: X   -6.661338e-17
dtype: float64
Mediana con escalamiento Z-score: X    0.0
dtype: float64
